In [1]:
import numpy as np

In [2]:
sign = np.vectorize(lambda x : 1 if x > 0 else -1)

In [3]:
prob = np.vectorize(lambda E,T: 1 / (1 + np.e ** (-E/T)))

In [28]:
class Hopfield :
    def __init__ (self,n_cells, memories, delta_t, temperature):
        self.n_cells = n_cells
        rows, cols = memories.shape
        self.weights = (memories.T * memories - np.mat(np.eye(cols))*rows) / float(cols)
        self.memories = memories
        self.delta_t = delta_t
        self.temperature = temperature
        
    def finish(self):
        self.weights = self.weights * (1/float(self.n_cells))
        
    def energy(self, state):
        return np.asscalar(-1/float(2) * state * self.weights * state.T)
    
    def nextState(self, state, temperature):
        if temperature == 0:
            return sign(self.weights * state.T).T
        else :
            probs = prob(self.weights * state.T, temperature).T
            sample_probs = np.mat(np.random.uniform(0,1,self.n_cells))
            return sign(probs-sample_probs)
        
    def findMinimum(self,state):
        temp = self.temperature
        actual_state = state
        equals = False
        while not equals and temp > 0.01:
            next_state = self.nextState(actual_state, temp)
            equals = np.all(np.equal(actual_state,next_state))
            actual_state = next_state
            temp = temp * self.delta_t
        return actual_state
    
    def error_bits(self, state):
        return 1*np.logical_not(np.equal(self.nextState(state,0), state))
    
    def isStable(self, error_per_bit_percentage):
        bit_errors_per_memory = [self.error_bits(memory) for memory in hn.memories]
        
        total_error_per_bit = np.mat(np.zeros(self.n_cells))
        
        for bit_errors in bit_errors_per_memory:
            total_error_per_bit = total_error_per_bit + bit_errors
        
        total_memories, size_memories = self.memories.shape

        error_per_bit_prob_stimated = total_error_per_bit.mean() / float(total_memories)
        
        return error_per_bit_prob_stimated < error_per_bit_percentage
    

In [39]:
N = 300
stable = True
memories = []
while stable:
    memories.append(sign(np.random.uniform(-1,1,N)))
    hn = Hopfield(n_cells=N,memories=np.mat(memories),delta_t=0.9, temperature=0.1)
    stable = hn.isStable(0.01)
print len(memories) / float(N)
  

0.193333333333


In [73]:
v = np.random.uniform(-1,1,3)
v_probs = prob(v,0.001)
print "V = ",v
print "probs = ", v_probs 
print sign(v_probs -np.random.uniform(0,1,3))

V =  [ 0.27107676 -0.03060197  0.29236612]
probs =  [  1.00000000e+00   5.12546861e-14   1.00000000e+00]
[ 1 -1  1]


In [116]:
v = np.array([0,0,0,0,1])
v[v>0].mean()

1.0

In [12]:
v = np.mat([1,2,3])
u = np.mat([1,2,4])
not np.equal(u,v)
#np.mat(map(lambda x : 1 if x else 0, np.equal(np.array(u),np.array(v))))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()